# Optimización lineal sobre el politopo de Birkhoff aplicado a modelo sencillo de elevadores

## Se definen funciones importantes

In [133]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import warnings

    
# Esta función dibuja el elevador

def printElevador(numero_Personas, numero_Elevadores, numero_Pisos, posicion_Personas, posicion_Elevadores):
    
    print("\n \n \n Modelo hiperealista del Burj Khalifa \n")
    
    
    n_personas_repetidas = np.amax(np.array([np.count_nonzero(posicion_Personas == i) for i in posicion_Personas]))   
    n_elevadores_repetidos = np.amax(np.array([np.count_nonzero(posicion_Elevadores == i) for i in posicion_Elevadores]))       
    for i in reversed(range(1,numero_Pisos+1)):
        
        k = len(str(i))
        n_perPiso = np.count_nonzero(posicion_Personas == i)
        elevs = ''
        for j in posicion_Elevadores:
            if j==i:
                elevs = elevs+'|████|'
            else:
                elevs = elevs+'|    |'
        print( "|"+'_'*5 +'_'*4*n_personas_repetidas + '_'*numero_Elevadores*6 + "| \n")
        print( "|"+' '*(4*(n_personas_repetidas-1)-3)+"Piso "+str(i)+' '*(6-k) + "|"  + elevs +"| \n")
        print( "|"+' '*(4+4*(n_personas_repetidas-n_perPiso))+' ●  '*n_perPiso + "|" +elevs+"| \n")
        print( "|"+' '*(4+4*(n_personas_repetidas-n_perPiso))+'¬█¬ '*n_perPiso + "|" +elevs+"| \n")
        print( "|"+' '*(4+4*(n_personas_repetidas-n_perPiso))+' n  '*n_perPiso+ "|" +elevs+"| \n")
    print("|"+'_'*(5+4*n_personas_repetidas+6*numero_Elevadores)+"|")
    
# Esta función resuelve el problema LP

def solElevador(numero_Personas, numero_Elevadores, numero_Pisos, posicion_Personas, posicion_Elevadores):
    warnings.filterwarnings('ignore')
    if numero_Personas>numero_Elevadores:
        numero_Esperando = numero_Personas-numero_Elevadores
        posicion_Esperando = posicion_Personas[numero_Esperando:]
        print("Hay ", numero_Esperando , " esperando a que se desocupen los ascensores.")
        numero_Personas = numero_Elevadores
        posicion_Personas = posicion_Personas[:numero_Elevadores]
       
    l = numero_Elevadores**2
    c = np.zeros(l)
    for i,posP in enumerate(posicion_Personas):
        c[numero_Elevadores*i:numero_Elevadores*(i+1)] = np.abs(posicion_Elevadores - posP)   
    
    A = np.zeros([2*numero_Elevadores,l])
    for i in range(numero_Elevadores):
        A[i, numero_Elevadores*i:numero_Elevadores*(i+1)] = np.ones(numero_Elevadores)
        A[i+numero_Elevadores, i::numero_Elevadores] = np.ones(numero_Elevadores)
    
    b = np.ones(2*numero_Elevadores)
    return linprog(c, A_eq=A, b_eq=b, bounds=(0,1), method='simplex')

def simuladorElevador():
    numero_Elevadores = int(input("Ingrese el número de elevadores: "))
    numero_Personas = int(input("Ingrese el número de personas: "))
    numero_Pisos = int(input("Ingrese el número de pisos del edificio: "))        
    posicion_Personas = np.array([np.random.randint(1,numero_Pisos+1) for i in range(numero_Personas)])
    posicion_Elevadores = np.array([np.random.randint(1,numero_Pisos+1) for i in range(numero_Elevadores)])        
    printElevador(numero_Personas, numero_Elevadores, numero_Pisos, posicion_Personas, posicion_Elevadores)           
    linPsolution = solElevador(numero_Personas, numero_Elevadores, numero_Pisos, posicion_Personas, posicion_Elevadores)
    M = np.reshape(linPsolution.x, [numero_Elevadores,numero_Elevadores])
    personas_orden = posicion_Personas[:numero_Elevadores]
    elevadores_orden = np.matmul(M,np.array(range(1,numero_Elevadores+1)).transpose())
    for i in range(numero_Elevadores):
        print("El elevador "+ str(int(elevadores_orden[i]))+ " recoge a la persona del piso "+ str(personas_orden[i]))
    
    print("Datos: \n")
    print("Posición de las personas recogidas: \n",personas_orden)
    print("\n Matriz de permutación de los elevadores: \n", M)
    print("\n Posición de todas las personas: \n",posicion_Personas)
    print("\n Posición de los elevadores: \n ", posicion_Elevadores)


In [135]:
simuladorElevador()

Ingrese el número de elevadores: 5
Ingrese el número de personas: 10
Ingrese el número de pisos del edificio: 5

 
 
 Modelo hiperealista del Burj Khalifa 

|_______________________________________________| 

|     Piso 5     ||    ||    ||    ||    ||    || 

|         ●   ●  ||    ||    ||    ||    ||    || 

|        ¬█¬ ¬█¬ ||    ||    ||    ||    ||    || 

|         n   n  ||    ||    ||    ||    ||    || 

|_______________________________________________| 

|     Piso 4     ||    ||████||    ||    ||████|| 

|             ●  ||    ||████||    ||    ||████|| 

|            ¬█¬ ||    ||████||    ||    ||████|| 

|             n  ||    ||████||    ||    ||████|| 

|_______________________________________________| 

|     Piso 3     ||    ||    ||    ||    ||    || 

|     ●   ●   ●  ||    ||    ||    ||    ||    || 

|    ¬█¬ ¬█¬ ¬█¬ ||    ||    ||    ||    ||    || 

|     n   n   n  ||    ||    ||    ||    ||    || 

|_______________________________________________| 

|     Piso 